In [ ]:
import typing
import qokit
import numpy as np
import networkx as nx
import scipy
import time
from matplotlib import pyplot as plt
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType
import qokit.maxcut as mc
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

"""
This file is created in the early stages of the project.
It may not reflect the latest research findings.
"""

def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None, simulator_name: str = "auto") -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name=simulator_name)
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim, simulator_name=simulator_name)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim, simulator_name=simulator_name)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_state(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"):
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_statevector(result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_expectation(result, preserve_state=True)


def get_overlap(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_overlap(result, preserve_state=True)

# Evaluates the objective function, and if expectations or measurements is passed in, appends to them
def inv_max_cut_objective(
    N: int, p: int, terms: TermsType, expectations: list[float] | None = None, measurements: list[float] | None = None, sim: QAOAFastSimulatorBase | None = None
) -> typing.Callable:
    def f(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if expectations != None:
            expectations.append((current_time, expectation))

        if measurements != None:
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))

        return -expectation

    return f

def QAOA_run(
    ising_model: TermsType,
    N: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    mixer: str = "x",  
    # If mixer == 'x', then uses the default Pauli X as the mixer
    # If mixer == 'xy', then uses the ring-XY as the mixer
    expectations: list[np.ndarray] | None = None,
    overlaps: list[np.ndarray] | None = None,
    simulator_name: str = "auto",
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_objective_function(ising_model, N, p, mixer, expectations, overlaps, simulator_name=simulator_name), init_freq, args=(), method=optimizer_method, options=optimizer_options
    )
    # the above returns a scipy optimization result object that has multiple attributes
    # result.x gives the optimal solutionsol.success #bool whether algorithm succeeded
    # result.message #message of why algorithms terminated
    # result.nfev is number of iterations used (here, number of QAOA calls)
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))
    
    if overlaps is not None:
        overlaps = list(map(make_time_relative, overlaps))

    gamma, beta = result.x[:p], result.x[p:]
    return {
        "gamma": gamma,
        "beta": beta,
        "state": get_state(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "expectation": get_expectation(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "overlap": get_overlap(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "runtime": end_time - start_time,  # measured in seconds
        "num_QAOA_calls": result.nfev,
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message
    }

In [ ]:
import matplotlib.pyplot as plt

def plot_expectation(expectations: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()

In [ ]:
# Generates a random graph with N vertices and runs MaxCut QAOA on this graph, plotting results
def run_experiment(seed: int, N: int, p: int) -> None:
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(N, 0.5, seed=seed)  # Random graph w/ 0.5 edge probability
    terms = mc.get_maxcut_terms(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    # Compute and print solution by brute force
    solution, solution_index = optimal_solution_with_index(sim)
    print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    # Random starting parameters
    init_gamma, init_beta = np.random.rand(2, p)
    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    # If you don't want expectation and measurement plots, do not pass in parameters!
    gamma, beta = optimize(N, terms, init_gamma, init_beta, expectations, measurements, sim)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")
    plot_expectation(expectations, N, p, start_time)